In [8]:

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1000362359804103118
]


In [9]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import time 
import scipy.io
import numpy as np
import tensorflow as tf
import h5py
import random
import matplotlib.pyplot as plt
from tensorflow.contrib import layers
from tensorflow.python.layers.core import Dense
import cv2
#print (cv2.__version__)

%matplotlib inline


In [10]:
def Architecture_gru(Input, outputs, is_training=True):
    with tf.variable_scope("Network"):
        
        with tf.variable_scope("CNN"):
            
            conv1=tf.layers.conv2d(Input,28,3,activation=tf.nn.relu,name='conv1',padding='same')
             #conv1=tf.layers.conv2d(Input,64,3,activation=tf.nn.relu,name='conv1',padding='same')
            pool1=tf.layers.max_pooling2d(conv1,2,[2,2],name='pool1')
            print('shape conv1',conv1.shape)
            print('shape pool1 [2,2]',pool1.shape)
            print('************************')
            
            conv2=tf.layers.conv2d(pool1,64,3,activation=tf.nn.relu,name='conv2',padding='same')
            #conv2=tf.layers.conv2d(pool1,128,3,activation=tf.nn.relu,name='conv2',padding='same')
            pool2=tf.layers.max_pooling2d(conv2,2,[2,1],name='pool2')
            print('shape conv2',conv2.shape)
            print('shape pool2 [2,1]',pool1.shape)
            print('************************')
            
            conv3=tf.layers.conv2d(pool2,128,3,activation=None,name='conv3',padding='same')
            #conv3=tf.layers.conv2d(pool2,256,3,activation=None,name='conv3',padding='same')
            b_norm3=tf.nn.relu(tf.layers.batch_normalization(conv3,training=is_training,name='batch-norm1'),name='b_norm3')
            print('shape conv3',conv3.shape)
            print('shape b_norm3',b_norm3.shape)
            print('************************')
            
            conv4=tf.layers.conv2d(b_norm3,256,3,activation=tf.nn.relu,name='conv4',padding='same')
            pool4=tf.layers.max_pooling2d(conv4,2,[2,1],name='pool4',padding='same')
            print('shape conv4',conv4.shape)
            print('shape pool4 [2,1]',pool4.shape)
            print('************************')
            
            conv5=tf.layers.conv2d(pool4,256,3,activation=tf.nn.relu,name='conv5',padding='same')
            pool5=tf.layers.max_pooling2d(conv5,2,[2,1],name='pool5',padding='same')
            print('shape conv5',conv5.shape)
            print('shape pool5 [2,1]',pool5.shape)
            print('************************')
            
            conv6=tf.layers.conv2d(pool5,256,3,activation=None,name='conv6',padding='same')
            b_norm6=tf.nn.relu(tf.layers.batch_normalization(conv6,training=is_training,name='batch-norm2'),name='b_norm6')
            print('shape conv6',conv6.shape)  
            print('shape b_norm6',b_norm6.shape)
            print('************************')
            
            conv7=tf.layers.conv2d(b_norm6,256,3,activation=tf.nn.relu,name='conv7',padding='same')
            pool7=tf.layers.max_pooling2d(conv7,2,[2,1],name='pool7',padding='same')
            print('shape conv7',conv7.shape)
            print('shape pool7[2,1]',pool7.shape)
            print('************************')
            
            conv8=tf.layers.conv2d(pool7,256,1,activation=None,name='conv8')
            #conv8=tf.layers.conv2d(pool7,256,2,activation=None,name='conv8')
            #conv8=tf.layers.conv2d(pool7,512,2,activation=None,name='conv8')
            b_norm8=tf.nn.relu(tf.layers.batch_normalization(conv8,training=is_training,name='batch-norm3'),name='b_norm8')
            print('shape conv8',conv8.shape)
            print('shape b_norm8',b_norm8.shape)
            print('************************')
            
            shape=b_norm8.get_shape().as_list()

            transposed=tf.transpose(b_norm8, perm=[0,2,1,3],name='transposed')
            conv_reshaped=tf.reshape(transposed, [shape[0], -1, shape[1]*shape[3]], name='reshaped')
            #print('shape conv_reshaped',conv_reshaped.shape)#shape conv_reshaped (32, 30, 256)
            
            
            
        num_units=256
       # num_units=512
        num_layers=3
        

       
        cell_stack_encoder=[]
        cell_stack_decoder=[]    
        
        with tf.variable_scope("Encoder"):
            for i in range(num_layers):
                cell_stack_encoder.append(tf.contrib.rnn.GRUCell(num_units))
            encoder_cell=tf.contrib.rnn.MultiRNNCell(cell_stack_encoder)
               
            _, encoder_state=tf.nn.dynamic_rnn(encoder_cell, conv_reshaped, dtype=tf.float32) 
           
            
            print('encoder_state', _.get_shape())
              
        with tf.variable_scope("Decoder"):
            for i in range(num_layers):
                cell_stack_decoder.append(tf.contrib.rnn.GRUCell(num_units))
            decoder_cell=tf.contrib.rnn.MultiRNNCell(cell_stack_decoder)
            
            dec_outputs,_=tf.nn.dynamic_rnn(decoder_cell,outputs, initial_state=encoder_state, dtype=tf.float32)
           
          
            
            logits=tf.layers.dense(dec_outputs,2, name='logits')
            
            return logits
            

In [16]:
def load_Images(Data_Train_Path):
    #main_path=[f for f in np.sort(os.listdir(Data_Train_Path)) if f.endswith('redraw')]
    main_path=[f for f in np.sort(os.listdir(Data_Train_Path)) if f.endswith('redraw')]
    
    imgs=[]
    for s in main_path:
        sub=Data_Train_Path+s
        sub_path= [f for f in np.sort(os.listdir(sub))if f.endswith('.tif')]
        for t in sub_path:
            im_path=sub + '/' +t
            img=cv2.imread(im_path,cv2.IMREAD_UNCHANGED)
            imgs.append(img)
           
    return (np.expand_dims(imgs, -1),t)


##*****************************************************************************************************
Save_folder ='/home/razen/thèse_2018/second_contrib/code/Besma_code_Recovery/recover_IRONOFF_caracters/LMCA_arabe_with_velocity/test_one_letter_recov_2018/fausse/recov/'

model_train_path = "/home/razen/tf-notebooks/logs_L3_64gru_3layer_256_LMCA/logs_L3_64gru_3layer_256_LMCA.ckpt-1"


number = 0
from tensorflow.python.framework import ops
ops.reset_default_graph()
tf.reset_default_graph()
global sess

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

sess = tf.Session (config=config)
graph = tf.get_default_graph()
import os

my_DataTest,f = load_Images('/home/razen/thèse_2018/second_contrib/code/Besma_code_Recovery/recover_IRONOFF_caracters/LMCA_arabe_with_velocity/test_one_letter_recov_2018/')



[Tot_Dat, row, col, depth] = np.shape(my_DataTest)


BatchSize_Test = 2

Inputs = tf.placeholder(tf.float32, shape = [BatchSize_Test, 64, 64, 1],name = 'Inputs')
outputs = tf.placeholder(tf.float32, shape = [BatchSize_Test, None, 2],name = 'outputs')

    
logits = Architecture_gru(Inputs, outputs)

saver = tf.train.Saver()
with tf.Session() as sess:
    
    saver.restore(sess, model_train_path)
   
    start = time.time()
    Test_loss = []

    
    for b in range(Tot_Dat//BatchSize_Test):
        batch_Test_input = my_DataTest[b*BatchSize_Test:(b + 1)*BatchSize_Test,:,:,:]
        print('b',b)
        print('debut b*BatchSize_Test',b*BatchSize_Test)
        print('fin batch_Test_input index',(b + 1)*BatchSize_Test)
        
        
        batch_Test_outputs = np.zeros(shape = [BatchSize_Test,1,2])
                  
        for ij in range(50):
               
            test_feed_dict = {Inputs:batch_Test_input, outputs:batch_Test_outputs}
            logits_Out = sess.run([logits], feed_dict = test_feed_dict)
            
            logits_Out = logits_Out[0]
            
            prediction = np.expand_dims(logits_Out[:,-1,:],axis = 1)
            batch_Test_outputs = np.hstack([batch_Test_outputs, prediction])
            

        ############################################
        for num in range(2):
            
            directory = Save_folder 
            try:
                os.stat(directory)
            except:
                os.mkdir(directory) 

            np.savetxt(  directory + 'Predict_' + str(num)+str(number) + '.txt', logits_Out[num,:,:])
            
            number = number + 1
       

shape conv1 (2, 64, 64, 28)
shape pool1 [2,2] (2, 32, 32, 28)
************************
shape conv2 (2, 32, 32, 64)
shape pool2 [2,1] (2, 32, 32, 28)
************************
shape conv3 (2, 16, 31, 128)
shape b_norm3 (2, 16, 31, 128)
************************
shape conv4 (2, 16, 31, 256)
shape pool4 [2,1] (2, 8, 31, 256)
************************
shape conv5 (2, 8, 31, 256)
shape pool5 [2,1] (2, 4, 31, 256)
************************
shape conv6 (2, 4, 31, 256)
shape b_norm6 (2, 4, 31, 256)
************************
shape conv7 (2, 4, 31, 256)
shape pool7[2,1] (2, 2, 31, 256)
************************
shape conv8 (2, 2, 31, 256)
shape b_norm8 (2, 2, 31, 256)
************************
encoder_state (2, 31, 256)
INFO:tensorflow:Restoring parameters from /home/razen/tf-notebooks/logs_L3_64gru_3layer_256_LMCA/logs_L3_64gru_3layer_256_LMCA.ckpt-1
b 0
debut b*BatchSize_Test 0
fin batch_Test_input index 2
